# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Female, VISl, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'F') & \
                             (sessions.full_genotype.str.find('wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
746083955,2019-10-03T00:00:00Z,726170935,brain_observatory_1.1,98.0,F,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,582,2216,6,"[VPM, TH, LGd, CA3, CA2, CA1, VISal, nan, grey..."
751348571,2019-10-03T00:00:00Z,732548380,brain_observatory_1.1,93.0,F,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,859,2217,6,"[grey, LGv, CA3, DG, CA1, VISl, nan, APN, POL,..."
798911424,2019-10-03T00:00:00Z,775876828,brain_observatory_1.1,110.0,F,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,825,2233,6,"[APN, TH, Eth, LP, DG, HPF, CA3, CA1, VISrl, n..."


In [7]:
session_id = filtered_sessions.index.values[0]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 582


,peak_channel_id,waveform_halfwidth,nn_hit_rate,waveform_amplitude,firing_rate,waveform_velocity_below,cumulative_drift,waveform_spread,amplitude_cutoff,waveform_recovery_slope,max_drift,L_ratio,local_index_unit,waveform_velocity_above,snr,nn_miss_rate,waveform_duration,presence_ratio,isolation_distance,isi_violations,d_prime,waveform_repolarization_slope,silhouette_score,waveform_PT_ratio,cluster_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951878213,849740502,0.151089,1.000,252.926895,28.717870,NaN,434.14,30.0,7.133207e-04,-0.127150,36.51,0.000004,0,-0.343384,4.780400,0.000000,0.206030,0.97,225.716963,0.000188,6.730306,0.627684,NaN,0.863448,0,NaN,0.0,3.370224,0.973333,2.664833,0.866667,2.017117,0.121025,0.114119,0.060497,0.015037,NaN,1.096409,0.700,0.08,4.0,0.202559,0.809524,0.377341,-1.000000,0.276139,135.0,30.0,0.158799,0.442171,2.761215e-04,0.166087,0.010034,NaN,4923,0.50,26.480777,0.053289,42.234275,0.042763,39.524292,NaN,0.1195,False,False,False,False,False,0.033915,False,0.051605,1.000000,0.004840,0.864463,0.007788,40,27,760647905,3,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.971918,1249.99883,True
951878313,849740524,0.164824,1.000,183.232725,49.122678,-0.412060,197.05,60.0,1.652670e-02,-0.162992,25.53,0.000003,8,0.343384,2.945178,0.000000,0.274707,0.99,155.205812,0.064869,7.590752,0.757797,0.152861,0.663130,8,NaN,200.0,8.546237,2.338352,1.891111,1.838703,1.708333,0.151282,0.031943,0.019111,0.032959,75.000,0.342966,0.379,0.08,1.0,0.385214,0.482870,0.396450,0.301913,0.361438,315.0,60.0,0.000154,0.000137,1.118768e-07,0.013595,0.000005,-30.000,4969,0.00,43.646607,42.258456,48.903658,72.895138,46.091468,NaN,0.0915,False,False,False,False,False,-0.016881,False,0.008061,0.002490,0.004696,0.007646,0.005235,160,59,760647905,14,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.971918,1249.99883,True
951878350,849740534,0.109883,0.996,149.232525,45.590934,-0.206030,190.89,70.0,1.202701e-03,-0.085155,20.46,0.001864,11,0.206030,3.341322,0.004095,0.206030,0.99,155.571595,0.004993,6.058813,0.636389,0.206435,0.272931,11,NaN,700.0,0.602806,1.037519,1.032773,0.976324,0.927689,0.285332,0.004176,0.006595,0.019456,14.286,0.702162,0.441,0.04,15.0,NaN,-0.241445,-0.226044,-0.217277,-0.192932,0.0,90.0,NaN,0.000292,1.323769e-03,0.605628,0.003479,2.857,4916,0.75,46.779123,47.560746,43.984073,47.308375,43.939267,NaN,0.0255,False,False,False,False,False,-0.108085,False,0.001015,0.000277,0.001427,0.005475,0.002367,200,27,760647905,19,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.971918,1249.99883,True
951878389,849740538,0.123618,0.990,175.529250,14.130594,-0.343384,287.04,70.0,1.216791e-02,-0.151065,21.36,0.000914,14,0.068677,3.166919,0.002185,0.206030,0.99,105.256300,0.029868,4.152507,0.747599,0.093003,0.461972,14,NaN,200.0,16.755080,3.961124,5.045263,7.933333,3.265306,0.356098,0.012501,0.048288,0.090532,10.000,0.894365,0.030,0.02,4.0,0.637658,0.657571,0.793651,0.489130,0.625903,180.0,60.0,0

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 8


,peak_channel_id,waveform_halfwidth,nn_hit_rate,waveform_amplitude,firing_rate,waveform_velocity_below,cumulative_drift,waveform_spread,amplitude_cutoff,waveform_recovery_slope,max_drift,L_ratio,local_index_unit,waveform_velocity_above,snr,nn_miss_rate,waveform_duration,presence_ratio,isolation_distance,isi_violations,d_prime,waveform_repolarization_slope,silhouette_score,waveform_PT_ratio,cluster_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951884504,849736154,NaN,0.766667,97.447935,9.608840,-0.343384,163.15,50.0,0.056563,NaN,57.15,0.034070,210,2.747069,1.591356,0.041667,1.538358,0.99,82.645995,0.008808,3.278342,0.033522,0.027395,1.297469,217,NaN,2100.0,5.174370,0.764228,1.148696,0.775758,3.714286,0.244868,0.067168,0.014237,0.161433,72.381,0.542828,0.0,0.02,8.0,0.194444,0.189255,0.674208,0.187302,-0.179487,0.0,90.0,0.578966,0.220582,0.023577,0.343314,0.616103,-13.333,4933,0.25,14.967350,9.405571,0.973976,14.505562,5.603982,NaN,0.1025,False,False,False,False,False,0.144458,False,0.014818,0.050822,0.115183,0.019874,0.052991,2060,11,760647911,205,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.924941,1249.996873,True
951884783,849736184,1.002680,0.903333,251.827485,12.046102,0.480737,83.69,70.0,0.000103,-0.141678,53.21,0.012446,237,0.618090,0.670429,0.023193,0.618090,0.95,108.312400,0.000000,4.253059,0.580924,0.011318,0.243576,244,NaN,600.0,7.105539,2.820449,2.181772,0.956463,2.356341,0.166656,0.038194,0.096716,0.173653,83.333,0.749008,0.0,0.04,8.0,0.385006,0.313920,0.037223,NaN,0.263103,180.0,30.0,0.001600,0.108473,0.878056,NaN,0.142819,43.333,4916,0.50,20.619223,7.673667,3.985240,15.788464,6.904228,NaN,0.2285,False,False,False,False,False,0.503322,False,0.076431,0.207546,0.196080,0.061801,0.254543,2220,43,760647911,220,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.924941,1249.996873,True
951884942,849736196,NaN,0.906000,194.675715,14.583070,0.137353,164.17,70.0,0.002081,-0.005547,40.02,0.007510,253,1.373534,0.904992,0.010907,1.359799,0.99,72.767268,0.048621,2.848197,0.098610,0.126389,3.759144,260,NaN,1100.0,7.492975,0.439913,3.956000,0.379487,2.934468,0.225985,0.065450,0.053548,0.078544,32.727,1.276830,0.0,0.02,1.0,0.085106,0.219048,0.484040,0.166667,0.460938,90.0,30.0,0.636641,0.004384,0.084077,0.432111,0.008896,17.273,4977,0.00,18.002438,21.928568,4.732179,20.685421,12.529525,NaN,0.1385,False,False,False,False,False,0.259068,False,0.018295,0.026298,0.109774,0.020897,0.066093,2280,59,760647911,226,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.924941,1249.996873,True
951885064,849736210,0.164824,0.827506,175.865430,2.319578,-0.343384,143.26,60.0,0.010081,-0.062927,58.16,0.008454,265,0.618090,1.817837,0.001386,0.673032,0.99,47.013649,0.050383,3.748553,0.473790,0.139398,0.865302,272,NaN,1300.0,5.594490,1.126275,2.931773,1.600000,1.637895,0.942187,0.093963,0.110641,0.095427,27.692,0.972498,0.0,0.04,4.0,0.344482,0.380952,0.490574,0.375000,0.653333,225.0,90.0,0.788995,0.42

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951884504     5.603982
951884783     6.904228
951884942    12.529525
951885064     2.171518
951885024     5.357521
951885161    23.957305
951885460    19.529673
951885668     1.405492
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

9.682405637985365


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951884504    14.967350
951884783    20.619223
951884942    18.002438
951885064     2.269433
951885024     1.761643
951885161     9.873039
951885460    12.862591
951885668     2.050750
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

10.300808419093098


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])